<a href="https://colab.research.google.com/github/Dh0t/NLP/blob/main/one_to_many.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Definisikan urutan input sampel sebagai array numerik tunggal
input_sequences = np.array([[1], [2], [3], [4], [5]])

# Definisikan urutan output yang sesuai dengan panjang variabel
output_sequences = [
    np.array([10, 20]),
    np.array([30, 40, 50]),
    np.array([60, 70]),
    np.array([80, 90, 100, 110]),
    np.array([120])
]

# Cetak urutan input dan output
print("Input Sequences:")
print(input_sequences)
print("\nOutput Sequences:")
for seq in output_sequences:
    print(seq)

Input Sequences:
[[1]
 [2]
 [3]
 [4]
 [5]]

Output Sequences:
[10 20]
[30 40 50]
[60 70]
[ 80  90 100 110]
[120]


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Input

# Define the model
model = Sequential()

# Add an Input layer to explicitly define the input shape
model.add(Input(shape=(1, 1)))

# Add an LSTM layer
# units: number of LSTM units
# input_shape: (timesteps, features) - This is now handled by the Input layer
# return_sequences: True because the TimeDistributed layer expects a sequence input
model.add(LSTM(50, return_sequences=True))

# Add a TimeDistributed Dense layer to output a sequence of predictions
model.add(TimeDistributed(Dense(50, activation='relu')))

# Add a final Dense layer to output a sequence of predictions
# The number of units here should match the expected output feature size per timestep.
# Based on the padding in the training step (padding to 10), let's set this to 10.
model.add(Dense(10))

# Display the model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 1, 50)          │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 1, 50)          │         2,550 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1, 10)          │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,460 (52.58 KB)

 Trainable params: 13,460 (52.58 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
import numpy as np # Import numpy again as it might not be available in this block's scope

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Pad the output sequences to a uniform length of 10 and reshape to (num_samples, 1, 10)
# to match the model's output shape (None, 1, 10)
padded_output_sequences_reshaped = np.zeros((len(output_sequences), 1, 10), dtype='float32')
for i, seq in enumerate(output_sequences):
    # Pad or truncate the sequence to length 10
    if len(seq) > 10:
        padded_seq = seq[:10]
    else:
        padded_seq = np.pad(seq, (0, 10 - len(seq)), 'constant')
    padded_output_sequences_reshaped[i, 0, :] = padded_seq

# Reshape input_sequences to match the model's expected input shape (num_samples, 1, 1)
input_sequences_reshaped = input_sequences.reshape((input_sequences.shape[0], 1, input_sequences.shape[1]))

# Train the model
history = model.fit(input_sequences_reshaped, padded_output_sequences_reshaped, epochs=100, batch_size=1, verbose=0)

# Store the training history
training_loss = history.history['loss']

# Print message and final training loss
print("Pelatihan Selesai.")
print(f"Final Training Loss: {training_loss[-1]:.4f}")

Pelatihan Selesai.
Final Training Loss: 474.2554


In [8]:
# Define a new input sequence
new_input_sequence = np.array([[6]])

# Reshape the new input sequence to match the model's expected input shape (batch_size, timesteps, features)
# The shape is (1, 1, 1) for a single input example
new_input_sequence_reshaped = new_input_sequence.reshape((1, 1, 1))

# Use the trained model to predict the output sequence
predicted_output = model.predict(new_input_sequence_reshaped)

# The prediction will be in the shape (batch_size, 1, 10). Flatten this output.
predicted_output_flat = predicted_output.flatten()

# Print the new input sequence and the corresponding predicted output sequence
print("New Input Sequence:")
print(new_input_sequence)
print("\nPredicted Output Sequence:")
print(predicted_output_flat)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
New Input Sequence:
[[6]]

Predicted Output Sequence:
[ 9.6682442e+01  6.4078529e+01  4.6185112e+01  3.2537781e+01
 -1.1775225e-02  3.7067644e-02  1.4545664e-01 -2.2508757e-01
 -1.5030907e-01  5.3136557e-02]
